# PL Standings for Single season

In [53]:
import os 
import json
import requests
import pandas as pd
from mysql import connector
from dotenv import load_dotenv

In [54]:
load_dotenv()

True

In [55]:
API_KEY = os.getenv("API_KEY")
API_HOST = os.getenv("API_HOST")
SEASON = 2021
LEAGUE_ID = 39

In [56]:
url = "https://v3.football.api-sports.io/standings"
headers = {
	"x-rapidapi-key": API_KEY,
	"x-rapidapi-host": API_HOST
}

querystring = {"season":SEASON,"league":LEAGUE_ID}

In [57]:
response = requests.request("GET", url, headers=headers, params=querystring)

In [58]:
payload = response.json()

In [59]:
payload

{'get': 'standings',
 'parameters': {'season': '2021', 'league': '39'},
 'errors': [],
 'results': 1,
 'paging': {'current': 1, 'total': 1},
 'response': [{'league': {'id': 39,
    'name': 'Premier League',
    'country': 'England',
    'logo': 'https://media.api-sports.io/football/leagues/39.png',
    'flag': 'https://media.api-sports.io/flags/gb-eng.svg',
    'season': 2021,
    'standings': [[{'rank': 1,
       'team': {'id': 50,
        'name': 'Manchester City',
        'logo': 'https://media.api-sports.io/football/teams/50.png'},
       'points': 93,
       'goalsDiff': 73,
       'group': 'Premier League',
       'form': 'WDWWW',
       'status': 'same',
       'description': 'Promotion - Champions League (Group Stage)',
       'all': {'played': 38,
        'win': 29,
        'draw': 6,
        'lose': 3,
        'goals': {'for': 99, 'against': 26}},
       'home': {'played': 19,
        'win': 15,
        'draw': 2,
        'lose': 2,
        'goals': {'for': 58, 'against': 15}

In [60]:
standings_list = payload['response'][0]['league']['standings'][0]

In [61]:
formatted_standings_list = json.dumps(standings_list, indent=4)
print(f"\n\n\n{formatted_standings_list}")




[
    {
        "rank": 1,
        "team": {
            "id": 50,
            "name": "Manchester City",
            "logo": "https://media.api-sports.io/football/teams/50.png"
        },
        "points": 93,
        "goalsDiff": 73,
        "group": "Premier League",
        "form": "WDWWW",
        "status": "same",
        "description": "Promotion - Champions League (Group Stage)",
        "all": {
            "played": 38,
            "win": 29,
            "draw": 6,
            "lose": 3,
            "goals": {
                "for": 99,
                "against": 26
            }
        },
        "home": {
            "played": 19,
            "win": 15,
            "draw": 2,
            "lose": 2,
            "goals": {
                "for": 58,
                "against": 15
            }
        },
        "away": {
            "played": 19,
            "win": 14,
            "draw": 4,
            "lose": 1,
            "goals": {
                "for": 41,
        

In [62]:
liverpool_points = standings_list[0]['points']

In [63]:
rows = []

column_names = [
    'season', 'position', 'team_id', 'team', 'played',
    'won', 'draw', 'lost', 'goals_for', 'goals_against', 
    'goal_diff', 'points', 'form'
    ]


for club in  standings_list:
    season             = 2021
    position           = club ['rank']
    team_id            = club['team']['id']
    team               = club['team']['name']
    played             = club['all']['played']
    won                = club['all']['win']
    draw               = club['all']['draw']
    lost               = club['all']['lose']
    goals_for          = club['all']['goals']['for']
    goals_against      = club['all']['goals']['against']
    goal_diff          = club['goalsDiff']
    points             = club['points']
    form               = club['form']
    
    tuple_of_club_records = (
        season, position,team_id, team, played, won, 
        draw, lost, goals_for, goals_against, 
        goal_diff, points, form
        )
    
    rows.append(tuple_of_club_records)

In [64]:
df = pd.DataFrame(rows, columns=column_names)

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   season         20 non-null     int64 
 1   position       20 non-null     int64 
 2   team_id        20 non-null     int64 
 3   team           20 non-null     object
 4   played         20 non-null     int64 
 5   won            20 non-null     int64 
 6   draw           20 non-null     int64 
 7   lost           20 non-null     int64 
 8   goals_for      20 non-null     int64 
 9   goals_against  20 non-null     int64 
 10  goal_diff      20 non-null     int64 
 11  points         20 non-null     int64 
 12  form           20 non-null     object
dtypes: int64(11), object(2)
memory usage: 2.2+ KB


In [66]:
MYSQL_HOST      = os.getenv('MYSQL_HOST')
MYSQL_PORT      = os.getenv('MYSQL_PORT')
MYSQL_USER      = os.getenv('MYSQL_USER')
MYSQL_PASSWORD  = os.getenv('MYSQL_PASSWORD')
MYSQL_DATABASE  = os.getenv('MYSQL_DATABASE')

In [67]:
server_conn = connector.connect(
    host               = MYSQL_HOST,
    port               = MYSQL_PORT,
    user               = MYSQL_USER,
    password           = MYSQL_PASSWORD,
    connection_timeout = 10,
    autocommit         = False,
    raise_on_warnings  = True
)

server_cur = server_conn.cursor()

In [68]:
print("Connected:", server_conn.is_connected())

Connected: True


In [69]:
server_cur.close()
server_conn.close()

In [70]:
db_connection = connector.connect(
    host               = MYSQL_HOST,
    port               = MYSQL_PORT,
    user               = MYSQL_USER,
    password           = MYSQL_PASSWORD,
    database           = MYSQL_DATABASE
)

print("Connected:", db_connection.is_connected())

Connected: True


In [71]:
cur = db_connection.cursor()

In [72]:
sql_table = "standings"
cur.execute("SHOW TABLES LIKE %s", (f"{sql_table}", ))

if cur.fetchone() is None:
    raise SystemExit(f"This table '{sql_table}' is NOT found...please create it...")
else:
    print(f"[SUCCESS] - This table '{sql_table}' exists! Continue to the next phase!")

[SUCCESS] - This table 'standings' exists! Continue to the next phase!


In [73]:
#user_team = "'Liverpool'; DROP TABLE standings"
#cur.execute("SELECT * FROM standings where team = %s", (user_team,))

In [74]:
# Start the UPSERT operation

table_cols = [
    'season', 'position', 'team_id', 'team', 'played',
    'won', 'draw', 'lost', 'goals_for', 'goals_against', 
    'goal_diff', 'points', 'form'
    ]

standings_df = df[table_cols]

In [75]:
standings_records_tuples = standings_df.itertuples(index=False, name=None)

list_of_standing_records_tuples = list(standings_records_tuples)

In [76]:
UPSERT_SQL = f""" 
INSERT INTO {sql_table}
(season, position,team_id, team, played, won, draw, lost, goals_for, goals_against, goal_diff, points, form)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) AS src
ON DUPLICATE KEY UPDATE
position         = src.position,
team             = src.team,
played           = src.played,
won              = src.won,
draw             = src.draw,
lost             = src.lost,
goals_for        = src.goals_for,
goals_against    = src.goals_against,
goal_diff        = src.goal_diff,
points           = src.points,
form             = src.form;
"""

In [77]:
no_of_rows_uploaded_to_mysql = len(list_of_standing_records_tuples)

In [78]:
try:
    cur.executemany(UPSERT_SQL, list_of_standing_records_tuples)
    db_connection.commit()
    print(f"[SUCCESS] - Upsert attempted for {no_of_rows_uploaded_to_mysql} rows!")
except Exception as e:
    db_connection.rollback()
    print(f"[ERROR] - Rolled back due to this ...: {e}")
finally:
    cur.close()
    db_connection.close()
    print("All database connections now closed. \n\nClean up completed.")

[SUCCESS] - Upsert attempted for 20 rows!
All database connections now closed. 

Clean up completed.
